In [1]:
import os
import sys
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torchvision.transforms as transforms

sys.path.append('../input/efficientnet/efficientnet-pytorch/EfficientNet-PyTorch/')
sys.path.append('../input/radampytorch/radam/RAdam/')

from efficientnet_pytorch import EfficientNet
from tqdm import tqdm_notebook
from PIL import Image
from radam import RAdam
from torch.utils.data import Dataset, DataLoader, ConcatDataset

In [2]:
train_df = pd.read_csv('../input/recursion-cellular-image-classification/train.csv')
test_df = pd.read_csv('../input/recursion-cellular-image-classification/test.csv')
sub = pd.read_csv('../input/recursion-cellular-image-classification/sample_submission.csv')

In [3]:
train_df_hepg = train_df.drop(train_df[train_df.experiment.str.find('HEPG') == -1].index).reset_index(drop=True)

In [4]:
import os
os.listdir('../input')

['radampytorch',
 'recursion-cellular-image-classification',
 'efficientnet',
 'fork-of-fork-of-fork-of-efficientnet-b3']

In [5]:
class CellDataset(Dataset):
    def __init__(self, df, img_dir, site=1, transforms=None):
        self.df = df
        self.img_dir = img_dir
        self.site = site
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        exp, well, plate = self.df.loc[idx,['experiment', 'well', 'plate']].values
        img_channels = [np.array(Image.open(os.path.join(self.img_dir,
                                             exp,
                                             f'Plate{plate}',
                                             f'{well}_s{self.site}_w{channel}.png')), 
                                          dtype=np.float32) for channel in range(1,7)]
        
        one_img = np.stack([channel for channel in img_channels],axis=2)
        
        if self.transforms is not None:
            one_img = self.transforms(one_img)
        if self.img_dir == '../input/recursion-cellular-image-classification/train/':
            return one_img, self.df.loc[idx,['sirna']].astype('int32').values
        else:
            return one_img     

In [6]:
# Augmentations for data
aug = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.485, 0.456, 0.456, 0.406, 0.406],
                                 std=[0.229, 0.229, 0.225, 0.225, 0.224, 0.224])
])

# Dataset & data loaders
dataset1 = CellDataset(df=train_df_hepg, img_dir='../input/recursion-cellular-image-classification/train/', transforms=aug)
dataset2 = CellDataset(df=train_df_hepg, img_dir='../input/recursion-cellular-image-classification/train/', transforms=aug, site=2)
dataset = ConcatDataset([dataset1, dataset2])
train_loader = DataLoader(dataset=dataset, batch_size=12, shuffle=True)

test_dataset = CellDataset(df=test_df, img_dir='../input/recursion-cellular-image-classification/test/', transforms=aug)
test_loader = DataLoader(dataset=test_dataset, batch_size=15, shuffle=False)

## Model

In [7]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=1108)

# Changes count input channels of our model
trained_kernel = model._conv_stem.weight
new_conv = nn.Sequential(nn.Conv2d(6, 40, kernel_size=(3,3), stride=(2,2), bias=False),
            nn.ZeroPad2d(padding=(0, 1, 0, 1)))
with torch.no_grad():
    new_conv[0].weight[:,:] = torch.stack([torch.mean(trained_kernel, 1)]*6, dim=1)
model._conv_stem = new_conv
model = model.cuda()

Downloading: "http://storage.googleapis.com/public-models/efficientnet/efficientnet-b3-5fb5a3c3.pth" to /tmp/.cache/torch/checkpoints/efficientnet-b3-5fb5a3c3.pth
100%|██████████| 47.1M/47.1M [00:00<00:00, 94.9MB/s]


Loaded pretrained weights for efficientnet-b3


In [8]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = RAdam(model.parameters(), lr=0.0001)

In [9]:
checkpoint = torch.load('../input/fork-of-fork-of-fork-of-efficientnet-b3/model.tar')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_num = checkpoint['epoch']

num_epochs = 6 + epoch_num  # add epochs previously done

In [10]:
# Train model
for epoch in range(epoch_num, num_epochs):
    for batch_i, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target[:,0].long().cuda()
        #print(data.shape)
        outputs = model(data)
        loss = criterion(outputs, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
    print('Epoch {} -> Train Loss: {:.4f}'.format(epoch+1, loss.item()))
    
torch.save({
            'epoch': num_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, 
    'model.tar')

Epoch 21 -> Train Loss: 1.4478
Epoch 22 -> Train Loss: 0.0574
Epoch 23 -> Train Loss: 0.5576
Epoch 24 -> Train Loss: 0.6003
Epoch 25 -> Train Loss: 0.4666
Epoch 26 -> Train Loss: 0.6480
